In [41]:
import os, gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
from pathlib import Path

In [42]:
%pwd

'c:\\Users\\takashi\\Documents\\Kaggle\\hms-harmful-brain-activity-classification\\working'

In [43]:
# !chcp 65001
# !dir
%ls

 Volume in drive C is Windows
 Volume Serial Number is 3E25-FCC2

 Directory of c:\Users\takashi\Documents\Kaggle\hms-harmful-brain-activity-classification\working

2024/02/26  11:34    <DIR>          .
2024/02/26  11:29    <DIR>          ..
2024/02/26  11:28           135,240 catboost-starter-lb-0-67.ipynb
2024/02/26  11:28            33,637 catboost-starter-ver3-inference.ipynb
2024/02/26  11:28           125,279 catboost-starter-ver3-train.ipynb
2024/02/28  13:10           159,237 eda-eeg.ipynb
2024/02/26  11:28         2,551,404 wavenet-starter-lb-0-66.ipynb
               5 File(s)      3,004,797 bytes
               2 Dir(s)  154,738,520,064 bytes free


In [51]:
temp = pl.read_parquet("../input/train_eegs/1001717358.parquet")
temp.head()

Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0
9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,0.0


In [ ]:
df_train = pl.read_csv("../input/train.csv")

In [125]:
PATH = '../input/train_eegs/'
files = os.listdir(PATH)
print(f'There are {len(files)} eeg parquet files')

all_eegs = {}
temp = pl.DataFrame()
for i, f in enumerate(files):
    if i%100 == 0: print(i, ', ', end = '')
    
    if i%5 ==3:
    # if (i < 1) or (eeg_id == 568657):
        temp_eeg = pl.read_parquet(f'{PATH}{f}')
        eeg_id = int(f.split('.')[0])
    

        # train データと、EEGデータにおける部分サンプルを対応させるための時間のキーをつくる
        # https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/468010
        
        # ex.
        # df_trainにおける eeg_label_offset_seconds = 0秒のとき
        # eeg全体における[0秒, 50秒]の50秒分の部分サンプルに対応する

        # df_trainの各行は、EEGの部分サンプル[T-25, T+25]の範囲に対応する
        # eeg_label_offset_secondsはその最初の時刻 T-25に等しい
        temp_offset = df_train.filter(pl.col("eeg_id") == eeg_id).select(["eeg_label_offset_seconds"]).to_series().to_list()

        
        # eegデータにおける、計測開始から各行までの秒数 seconds_eeg
        t = 1/200         # EEGのサンプリング周波数は200Hzなので、1行は1/200秒ごとの計測値を表す  
        seconds = [i*t for i in range(len(temp_eeg))]
        temp_eeg = temp_eeg.with_columns(pl.Series(seconds).alias("seconds_eeg")) 
        
        # eegデータにおける計測時刻 seconds_eeg から25秒引いたときの時刻 seconds_eeg_offset
        # = df_trainにおけるeeg_label_offset_seconds
        temp_eeg = temp_eeg.with_columns((pl.col("seconds_eeg") - 25).alias("seconds_eeg_offset"))


        # 差分特徴量を作る
        temp_eeg = temp_eeg.with_columns(
            (pl.col("Fp1") - pl.col("T3")).alias("LL_Fp1-T3"),
            (pl.col("T3") - pl.col("O1")).alias("LL_T3-O1"),
            (pl.col("Fp1") - pl.col("C3")).alias("LP_Fp1-C3"),
            (pl.col("C3") - pl.col("O1")).alias("LP_C3-O1"),
            (pl.col("Fp2") - pl.col("C4")).alias("RP_Fp2-C4"),
            (pl.col("C4") - pl.col("O2")).alias("RP_C4-O2"),
            (pl.col("Fp2") - pl.col("T4")).alias("RR_Fp2-T4"),
            (pl.col("T4") - pl.col("O2")).alias("RR_T4-O2"),
        )

        # 差分特徴量の集計特徴量をつくる
        window = 4000 # 20秒窓 (20秒 / 1レコード1/200秒 = 4000レコード)
        temp_eeg = temp_eeg.with_columns(
            # LL: Fp1 - T3
            pl.col("LL_Fp1-T3").rolling_mean(window_size=window).alias("mean20s_LL_Fp1-T3"),
            pl.col("LL_Fp1-T3").rolling_max(window_size=window).alias("min20s_LL_Fp1-T3"),
            pl.col("LL_Fp1-T3").rolling_min(window_size=window).alias("max20s_LL_Fp1-T3"),
            pl.col("LL_Fp1-T3").rolling_std(window_size=window).alias("std20s_LL_Fp1-T3"),
            (pl.col("LL_Fp1-T3").rolling_max(window_size=window) - pl.col("LL_Fp1-T3").rolling_min(window_size=window)).alias("max-min_20s_LL_Fp1-T3"),

            # LL: Fp1 - T3
            pl.col("LL_T3-O1").rolling_mean(window_size=window).alias("mean20s_LL_T3-O1"),
            pl.col("LL_T3-O1").rolling_max(window_size=window).alias("min20s_LL_T3-O1"),
            pl.col("LL_T3-O1").rolling_min(window_size=window).alias("max20s_LL_T3-O1"),
            pl.col("LL_T3-O1").rolling_std(window_size=window).alias("std20s_LL_T3-O1"),
            (pl.col("LL_T3-O1").rolling_max(window_size=window) - pl.col("LL_T3-O1").rolling_min(window_size=window)).alias("max-min_20s_LL_T3-O1"),


            # LP: Fp1 - C3
            pl.col("LP_Fp1-C3").rolling_mean(window_size=window).alias("mean20s_LP_Fp1-C3"),
            pl.col("LP_Fp1-C3").rolling_max(window_size=window).alias("min20s_LP_Fp1-C3"),
            pl.col("LP_Fp1-C3").rolling_min(window_size=window).alias("max20s_LP_Fp1-C3"),
            pl.col("LP_Fp1-C3").rolling_std(window_size=window).alias("std20s_LP_Fp1-C3"),
            (pl.col("LP_Fp1-C3").rolling_max(window_size=window) - pl.col("LP_Fp1-C3").rolling_min(window_size=window)).alias("max-min_20s_LP_Fp1-C3"),

            # LP: C3 - O1
            pl.col("LP_C3-O1").rolling_mean(window_size=window).alias("mean20s_LP_C3-O1"),
            pl.col("LP_C3-O1").rolling_max(window_size=window).alias("min20s_LP_C3-O1"),
            pl.col("LP_C3-O1").rolling_min(window_size=window).alias("max20s_LP_C3-O1"),
            pl.col("LP_C3-O1").rolling_std(window_size=window).alias("std20s_LP_C3-O1"),
            (pl.col("LP_C3-O1").rolling_max(window_size=window) - pl.col("LP_C3-O1").rolling_min(window_size=window)).alias("max-min_20s_LP_C3-O1"),


            # RP: Fp1 - C4
            pl.col("RP_Fp2-C4").rolling_mean(window_size=window).alias("mean20s_RP_Fp2-C4"),
            pl.col("RP_Fp2-C4").rolling_max(window_size=window).alias("min20s_RP_Fp2-C4"),
            pl.col("RP_Fp2-C4").rolling_min(window_size=window).alias("max20s_RP_Fp2-C4"),
            pl.col("RP_Fp2-C4").rolling_std(window_size=window).alias("std20s_RP_Fp2-C4"),
            (pl.col("RP_Fp2-C4").rolling_max(window_size=window) - pl.col("RP_Fp2-C4").rolling_min(window_size=window)).alias("max-min_20s_RP_Fp2-C4"),

            # RP: C4 - O2
            pl.col("RP_C4-O2").rolling_mean(window_size=window).alias("mean20s_RP_C4-O2"),
            pl.col("RP_C4-O2").rolling_max(window_size=window).alias("min20s_RP_C4-O2"),
            pl.col("RP_C4-O2").rolling_min(window_size=window).alias("max20s_RP_C4-O2"),
            pl.col("RP_C4-O2").rolling_std(window_size=window).alias("std20s_RP_C4-O2"),
            (pl.col("RP_C4-O2").rolling_max(window_size=window) - pl.col("RP_C4-O2").rolling_min(window_size=window)).alias("max-min_20s_RP_C4-O2"),


            # RR: Fp2 - T4
            pl.col("RR_Fp2-T4").rolling_mean(window_size=window).alias("mean20s_RR_Fp2-T4"),
            pl.col("RR_Fp2-T4").rolling_max(window_size=window).alias("min20s_RR_Fp2-T4"),
            pl.col("RR_Fp2-T4").rolling_min(window_size=window).alias("max20s_RR_Fp2-T4"),
            pl.col("RR_Fp2-T4").rolling_std(window_size=window).alias("std20s_RR_Fp2-T4"),
            (pl.col("RR_Fp2-T4").rolling_max(window_size=window) - pl.col("RR_Fp2-T4").rolling_min(window_size=window)).alias("max-min_20s_RR_Fp2-T4"),

            # RR: T4 - O2
            pl.col("RR_T4-O2").rolling_mean(window_size=window).alias("mean20s_RR_T4-O2"),
            pl.col("RR_T4-O2").rolling_max(window_size=window).alias("min20s_RR_T4-O2"),
            pl.col("RR_T4-O2").rolling_min(window_size=window).alias("max20s_RR_T4-O2"),
            pl.col("RR_T4-O2").rolling_std(window_size=window).alias("std20s_RR_T4-O2"),
            (pl.col("RR_T4-O2").rolling_max(window_size=window) - pl.col("RR_T4-O2").rolling_min(window_size=window)).alias("max-min_20s_RR_T4-O2"),
        )



        # 集計特徴量とEKG以外drop
        temp_eeg = temp_eeg.drop(['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2'])
        
        # seconds_eeg_offsetが、eeg_label_offset_secondsのリストtemp_offsetに含まれるレコードのみ抽出
        temp_eeg = temp_eeg.filter(pl.col("seconds_eeg_offset").is_in(temp_offset))

        # eeg_id を追加
            # 型        
        temp_eeg = temp_eeg.with_columns(pl.lit(str(eeg_id)).alias("eeg_id"))

        # 全ての
        temp = pl.concat([temp, temp_eeg], how = 'vertical')

    else:
        continue
        
            


There are 17300 eeg parquet files
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 13300 , 13400 

In [128]:
temp = temp.with_columns(pl.col("eeg_id").cast(pl.Int64).alias("eeg_id"))
temp.head()

EKG,seconds_eeg,seconds_eeg_offset,LL_Fp1-T3,LL_T3-O1,LP_Fp1-C3,LP_C3-O1,RP_Fp2-C4,RP_C4-O2,RR_Fp2-T4,RR_T4-O2,mean20s_LL_Fp1-T3,min20s_LL_Fp1-T3,max20s_LL_Fp1-T3,std20s_LL_Fp1-T3,max-min_20s_LL_Fp1-T3,mean20s_LL_T3-O1,min20s_LL_T3-O1,max20s_LL_T3-O1,std20s_LL_T3-O1,max-min_20s_LL_T3-O1,mean20s_LP_Fp1-C3,min20s_LP_Fp1-C3,max20s_LP_Fp1-C3,std20s_LP_Fp1-C3,max-min_20s_LP_Fp1-C3,mean20s_LP_C3-O1,min20s_LP_C3-O1,max20s_LP_C3-O1,std20s_LP_C3-O1,max-min_20s_LP_C3-O1,mean20s_RP_Fp2-C4,min20s_RP_Fp2-C4,max20s_RP_Fp2-C4,std20s_RP_Fp2-C4,max-min_20s_RP_Fp2-C4,mean20s_RP_C4-O2,min20s_RP_C4-O2,max20s_RP_C4-O2,std20s_RP_C4-O2,max-min_20s_RP_C4-O2,mean20s_RR_Fp2-T4,min20s_RR_Fp2-T4,max20s_RR_Fp2-T4,std20s_RR_Fp2-T4,max-min_20s_RR_Fp2-T4,mean20s_RR_T4-O2,min20s_RR_T4-O2,max20s_RR_T4-O2,std20s_RR_T4-O2,max-min_20s_RR_T4-O2,eeg_id
f32,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
191.660004,25.0,0.0,323.379974,-108.759979,189.460007,25.160004,-43.889999,-83.440002,777.929993,-905.26001,412.67749,4634.339844,0.0,915.040771,4634.339844,-127.406128,0.0,-1539.919922,292.541199,1539.919922,297.551331,3225.75,0.0,650.571655,3225.75,-12.279777,116.329987,-269.379883,42.236317,385.709869,11.759129,228.540009,-144.449951,42.865955,372.98996,-252.892807,0.0,-2786.389893,566.029724,2786.389893,266.312775,1349.469971,0.0,462.041687,1349.469971,-507.446899,0.0,-3706.030029,956.973328,3706.030029,1001717358
131.330002,29.0,4.0,55.320004,-128.419998,81.449997,-154.550003,33.689999,17.609999,520.340027,-469.040009,449.258148,4634.339844,-12.08,902.251709,4646.419922,-138.697495,74.450005,-1539.919922,289.550018,1614.369873,323.572083,3225.75,-22.5,641.607727,3248.25,-13.011845,158.76001,-269.379883,49.844265,428.139893,31.754271,261.399994,-144.449951,60.888275,405.849945,-271.410187,91.800003,-2786.389893,560.559875,2878.189941,424.303894,1349.469971,0.0,480.897888,1349.469971,-663.961243,0.0,-3706.030029,926.889893,3706.030029,1001717358
140.070007,37.0,12.0,23.360001,-8.020004,-25.400002,40.739998,159.150009,4.69,458.25,-294.410004,470.352783,4634.339844,-88.950005,892.572388,4723.290039,-147.161682,124.440002,-1539.919922,287.613861,1664.359863,336.189972,3225.75,-144.429993,636.141541,3370.179932,-12.998888,202.220001,-269.379883,63.660961,471.599884,72.649086,261.399994,-144.449951,68.670013,405.849945,-276.438446,134.570007,-2786.389893,559.632202,2920.959961,636.892883,1349.469971,0.0,350.520569,1349.469971,-840.682922,0.0,-3706.030029,821.597656,3706.030029,1001717358
-70.300003,67.0,42.0,167.830002,-17.989998,142.240005,7.599998,102.649994,6.710003,183.730011,-74.370003,53.022327,221.830002,-151.080002,55.918518,372.910004,-22.694662,129.100006,-190.410004,53.07122,319.51001,31.672546,260.149994,-176.779999,63.383339,436.929993,-1.345063,168.059998,-231.380005,65.765457,399.440002,107.228745,275.700012,-72.509995,57.135098,348.210022,6.13805,151.139999,-242.290009,63.278687,393.429993,229.746231,483.440002,0.149998,87.56395,483.290009,-116.381439,95.869995,-368.970001,71.364052,464.839996,1001717358
-127.050003,105.0,80.0,121.889999,15.77,74.920006,62.740002,131.899994,32.860001,102.580002,62.18,39.584751,236.180008,-149.619995,65.510597,385.799988,-18.363068,173.779999,-243.619995,66.816132,417.399994,21.645119,224.449982,-160.419998,68.567253,384.869995,-0.423445,210.040009,-243.540009,77.368248,453.580017,88.804695,236.12999,-42.09,48.630436,278.220001,0.907417,172.419998,-261.070007,61.006084,433.48999,86.561203,336.519989,-155.779999,79.498024,492.299988,3.148824,223.270004,-229.730011,73.906242,453.0,1001717358


In [138]:
FEATURES = temp.drop(["seconds_eeg", "seconds_eeg_offset", "eeg_id"]).columns

In [139]:
FEATURES

['EKG',
 'LL_Fp1-T3',
 'LL_T3-O1',
 'LP_Fp1-C3',
 'LP_C3-O1',
 'RP_Fp2-C4',
 'RP_C4-O2',
 'RR_Fp2-T4',
 'RR_T4-O2',
 'mean20s_LL_Fp1-T3',
 'min20s_LL_Fp1-T3',
 'max20s_LL_Fp1-T3',
 'std20s_LL_Fp1-T3',
 'max-min_20s_LL_Fp1-T3',
 'mean20s_LL_T3-O1',
 'min20s_LL_T3-O1',
 'max20s_LL_T3-O1',
 'std20s_LL_T3-O1',
 'max-min_20s_LL_T3-O1',
 'mean20s_LP_Fp1-C3',
 'min20s_LP_Fp1-C3',
 'max20s_LP_Fp1-C3',
 'std20s_LP_Fp1-C3',
 'max-min_20s_LP_Fp1-C3',
 'mean20s_LP_C3-O1',
 'min20s_LP_C3-O1',
 'max20s_LP_C3-O1',
 'std20s_LP_C3-O1',
 'max-min_20s_LP_C3-O1',
 'mean20s_RP_Fp2-C4',
 'min20s_RP_Fp2-C4',
 'max20s_RP_Fp2-C4',
 'std20s_RP_Fp2-C4',
 'max-min_20s_RP_Fp2-C4',
 'mean20s_RP_C4-O2',
 'min20s_RP_C4-O2',
 'max20s_RP_C4-O2',
 'std20s_RP_C4-O2',
 'max-min_20s_RP_C4-O2',
 'mean20s_RR_Fp2-T4',
 'min20s_RR_Fp2-T4',
 'max20s_RR_Fp2-T4',
 'std20s_RR_Fp2-T4',
 'max-min_20s_RR_Fp2-T4',
 'mean20s_RR_T4-O2',
 'min20s_RR_T4-O2',
 'max20s_RR_T4-O2',
 'std20s_RR_T4-O2',
 'max-min_20s_RR_T4-O2']